# Step 3: Perform fit

As explained in the [previous step](2_generate_data), an [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) object behaves just like a mathematical function that takes a set of data points as an argument and returns a list of intensities (real numbers). At this stage, we want to optimize the parameters of the intensity model, so that it matches the distribution of our data sample. This is what we call 'performing a fit'.

```{note}
We first load the relevant data from the previous steps.
```

In [ ]:
import numpy as np
from expertsystem import io

from tensorwaves.physics.helicity_formalism.amplitude import IntensityBuilder
from tensorwaves.physics.helicity_formalism.kinematics import HelicityKinematics

In [ ]:
model = io.load_amplitude_model("amplitude_model_helicity.yml")
kinematics = HelicityKinematics.from_model(model)
data_sample = np.load("data_sample.npy")
phsp_sample = np.load("phsp_sample.npy")
intensity_builder = IntensityBuilder(model.particles, kinematics, phsp_sample)
intensity = intensity_builder.create_intensity(model)
phsp_set = kinematics.convert(phsp_sample)
data_set = kinematics.convert(data_sample)

## 3.1 Define estimator

To perform a fit, you need to define an **estimator**. An estimator is a measure for the discrepancy between the intensity model and the data distribution to which you fit. In PWA, we usually use an *unbinned negative log likelihood estimator*, which can be created as follows with the [tensorwaves.estimator](tensorwaves.estimator) module:

In [ ]:
from tensorwaves.estimator import UnbinnedNLL

estimator = UnbinnedNLL(intensity, data_set)

The parameters that the [UnbinnedNLL](tensorwaves.estimator.UnbinnedNLL) carries, have been taken from the [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) object and you'll recognize them from the YAML recipe file that we created in [Step 1](1_create_model):

In [ ]:
estimator.parameters

## 3.2 Optimize the intensity model

Now it's time to perform the fit! This is the part where the TensorFlow backend comes into play. For this reason, we suppress a few warnings:

In [ ]:
import tensorflow as tf

tf.get_logger().setLevel("ERROR")

Starting the fit itself is quite simple: just create an optimizer instance of your choice, here [Minuit2](https://root.cern.ch/doc/master/Minuit2Page.html), and call its [optimize](tensorwaves.optimizer.minuit.Minuit2.optimize) method to start the fitting process. Notice that the [optimize](tensorwaves.optimizer.minuit.Minuit2.optimize) method requires a second argument. This is a mapping of parameter names that you want to fit to their initial values.

Let's first select a few of the parameters that we saw in {ref}`Step 3.1 <usage/3_perform_fit:3.1 Define estimator>` and feed them to the optimizer to run the fit. Notice that we modify the parameters slightly to make the fit more interesting (we are running using a data sample that was generated with this very amplitude model after all).

In [ ]:
initial_parameters = {
    "Phase_J/psi(1S)_to_f(0)(1500)_0+gamma_1;f(0)(1500)_to_pi0_0+pi0_0;": 0.0,
    "Width_f(0)(500)": 0.12,
    "Position_f(0)(1710)": 1.8,
    "Width_f(0)(1710)": 0.3,
}

Recall that an [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) object is really just a [Function](tensorwaves.interfaces.Function) that computes the intensity for a certain dataset. This can be seen now nicely when we use these intensities as weights on the phase space sample and plot it together with the original dataset. Here, we look at the invariant mass distribution projection of the final states `3` and `4`, which, {ref}`as we saw before <usage/2_generate_data:2.4 Visualize kinematic variables>`, are the final state particles $\pi^0,\pi^0$.

Don't forget to use [update_parameters](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF.update_parameters) first!

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def compare_model(
    variable_name, data_set, phsp_set, intensity_model, transform=lambda p: p, bins=100
):
    data = transform(data_set[variable_name])
    phsp = transform(phsp_set[variable_name])
    intensities = intensity_model(phsp_set)
    plt.hist(data, bins=bins, alpha=0.5, label="data", density=True)
    plt.hist(
        phsp,
        weights=intensities,
        bins=bins,
        histtype="step",
        color="red",
        label="initial fit model",
        density=True,
    )
    plt.legend()

In [ ]:
intensity.update_parameters(initial_parameters)
compare_model("mSq_3+4", data_set, phsp_set, intensity, np.sqrt)

Finally, we create an [Optimizer](tensorwaves.interfaces.Optimizer) to [optimize](tensorwaves.optimizer.minuit.Minuit2.optimize) the model (which is embedded in the [Estimator](tensorwaves.interfaces.Estimator). Only the parameters that are given to the [optimize](tensorwaves.optimizer.minuit.Minuit2.optimize) method are optimized.

Here, we choose to use {class}`.Minuit2`, which is the most common optimizer choice in high-energy physics. Notice that the {class}`.Minuit2` class allows one to list {mod}`~tensorwaves.optimizer.callbacks`. These are called during the {meth}`~.Optimizer.optimize` method. Here, we use {class}`.CallbackList` to 'stack' several callbacks together.

```{toggle}
To define your own callback, create a class that inherits from the {class}`~.Callback` class and feed it to the {class}`.Minuit2` constructor.
```

In [ ]:
from tensorwaves.optimizer.callbacks import (
    CallbackList,
    CSVSummary,
    TFSummary,
    YAMLSummary,
)
from tensorwaves.optimizer.minuit import Minuit2

minuit2 = Minuit2(
    callback=CallbackList(
        [
            TFSummary(),
            YAMLSummary("current_fit_result.yaml"),
            CSVSummary("fit_traceback.csv", step_size=2),
        ]
    )
)
result = minuit2.optimize(estimator, initial_parameters)
result

```{admonition} Computation time ― around a minute in this case
---
class: dropdown
---
The computation time depends on the complexity of the model, number of data events, the size of the phase space sample, and the number of free parameters. This model is rather small and has but a few free parameters, so the optimization shouldn't take more than a few minutes.
```

As can be seen, the values of the optimized parameters in the result are again comparable to the original values we saw in {ref}`usage/3_perform_fit:3.1 Define estimator`:

In [ ]:
optimized_parameters = result["parameter_values"]
optimized_parameters

## 3.3 Export and import

In {ref}`usage/3_perform_fit:3.2 Optimize the intensity model`, we initialized {obj}`.Minuit2` with some callbacks that are {class}`.Loadable`. Such callback classes offer the possibility to {meth}`.Loadable.load_latest_parameters`, so you can pick up the optimize process in case it crashes or if you pause it. Loading the latest parameters goes as follows:

In [ ]:
latest_parameters = YAMLSummary.load_latest_parameters("current_fit_result.yaml")
latest_parameters

To restart the fit with the latest parameters, simply rerun as before.

In [ ]:
minuit2 = Minuit2()
minuit2.optimize(estimator, latest_parameters)

Lo and behold: the parameters were already optimized, so the fit converged faster!

## 3.4 Visualize

### Plot optimized model

Using the same method as above, we renew the parameters of the [IntensityTF](tensorwaves.physics.helicity_formalism.amplitude.IntensityTF) and plot it again over the data distribution.

In [ ]:
intensity.update_parameters(latest_parameters)
compare_model("mSq_3+4", data_set, phsp_set, intensity, np.sqrt)

### Analyze optimization process

Note that {ref}`in Step 3.2 <usage/3_perform_fit:3.2 Optimize the intensity model>`, we initialized {class}`.Minuit2` with a {class}`.TFSummary` callback as well. Its output files provide a nice, interactive representation of the fit process and can be viewed with [TensorBoard](https://www.tensorflow.org/tensorboard/get_started) as follows:

````{tabbed} Terminal
```bash
tensorboard --logdir logs
```
````

````{tabbed} Python
```python
import tensorboard as tb

tb.notebook.list()  # View open TensorBoard instances
tb.notebook.start(args_string="--logdir logs")
```
See more info [here](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks#tensorboard_in_notebooks)
````

````{tabbed} Jupyter notebook
```ipython
%load_ext tensorboard
%tensorboard --logdir logs
```
See more info [here](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks#tensorboard_in_notebooks)
````

An alternative would be to use the output of the {class}`.CSVSummary` callback. Here's an example:

In [ ]:
import pandas as pd

fit_traceback = pd.read_csv("fit_traceback.csv")

In [ ]:
fit_traceback.plot("function_call", "estimator_value");

In [ ]:
fit_traceback.plot(
    "function_call",
    [
        "Phase_J/psi(1S)_to_f(0)(1500)_0+gamma_1;f(0)(1500)_to_pi0_0+pi0_0;",
        "Position_f(0)(1710)",
        "Width_f(0)(1710)",
    ],
);